# Задание 1. Загрузка данных
Изучить представленный набор данных на основе описания его столбцов в файле “horse_data.names” , загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.


In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("D:\\Нетология python\\Статистика_в_Python\\поиск_аномалий_в_данных_и_работа_с_пропущенными_значениями\\horse_data.csv", usecols=[1,2,4,5,6,7,11,23],names=['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome'])


In [5]:
df.head()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1,530101,66,28,3,3,4,2
1,1,534817,88,20,?,?,4,2
2,1,530334,40,24,1,1,3,2
3,9,5290409,164,84,4,1,4,1
4,1,530255,104,35,?,?,?,2


# Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [29]:
statistics = []

for column in df.columns:
    if df[column].dtype in ['int64', 'float64']:
        sr = df[column]
        min_ = sr.min()
        max_ = sr.max()
        range_ = max_ - min_
        disp = sr.var()
        mean = sr.mean()
        median = sr.median()
        mode = int(sr.mode()[0])

        statistics.append({'Column': column, 'min_': min_, 'max_': max_, 'range_': range_, 'disp': disp, 'mean': mean, 'median': median, 'mode': mode})

statistics_df = pd.DataFrame(statistics)

In [30]:
statistics_df

,Column,min_,max_,range_,disp,mean,median,mode
0,surgery?,1,9,8,4.726154e+00,1.640000e+00,1.0,1
1,Age,518476,5305629,4787153,2.340291e+12,1.085889e+06,530305.5,527544
2,outcome,1,2,1,2.320959e-01,1.363333e+00,1.0,1


In [32]:

outliers = []

for column in df.columns:
    if df[column].dtype in ['int64', 'float64']:
        sr = df[column]

        if sr.astype(str).str.isnumeric().all():
            q1 = sr.quantile(0.25)
            q3 = sr.quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr

            column_outliers = sr[(sr < lower_bound) | (sr > upper_bound)]
            outliers_count = len(column_outliers)
            outliers.append({'Column': column, 'Outliers count': outliers_count, 'Outliers': column_outliers.values})
        else:
            continue

outliers_df = pd.DataFrame(outliers)



In [33]:

outliers_df.head()

,Column,Outliers count,Outliers
0,surgery?,24,"[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ..."
1,Age,36,"[5290409, 5299629, 5291329, 5301219, 5290481, ..."
2,outcome,0,[]


# Вывод.
Производят вычислительные операции, и если данных нет или знаки, методы падают с ошибкой, и в итоге пропускаем много данных.
из 8 колонок остается только 3 

# ==============================================================

# Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [58]:
df = pd.read_csv("D:\\Нетология python\\Статистика_в_Python\\поиск_аномалий_в_данных_и_работа_с_пропущенными_значениями\\horse_data.csv", usecols=[1,2,4,5,6,7,11,23], names=['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome'])

# Применение pd.to_numeric() к каждой колонке по отдельности
df['surgery?'] = pd.to_numeric(df['surgery?'], errors='coerce')
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['rectal temperature'] = pd.to_numeric(df['rectal temperature'], errors='coerce')
df['pulse'] = pd.to_numeric(df['pulse'], errors='coerce')
df['respiratory rate'] = pd.to_numeric(df['respiratory rate'], errors='coerce')
df['temperature of extremities'] = pd.to_numeric(df['temperature of extremities'], errors='coerce')
df['pain'] = pd.to_numeric(df['pain'], errors='coerce')
df['outcome'] = pd.to_numeric(df['outcome'], errors='coerce')

# Подсчет количества пропусков, где значение не число
non_numeric_counts = df.isna().sum()

# Вывод количества пропусков, где значение не число
non_numeric_counts




surgery?                       0
Age                            0
rectal temperature            24
pulse                         58
respiratory rate              56
temperature of extremities    69
pain                          44
outcome                        0
dtype: int64

In [59]:
import pandas as pd

df = pd.read_csv("D:\\Нетология python\\Статистика_в_Python\\поиск_аномалий_в_данных_и_работа_с_пропущенными_значениями\\horse_data.csv", usecols=[1,2,4,5,6,7,11,23], names=['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome'])
null_counts = df.isna().sum()
# Замена знаков "?" на 0
df.replace("?", 0, inplace=True)

# Подсчет количества пропусков по столбцам


# Проверка типа данных и преобразование строковых значений в числа, если необходимо
for column in df.columns:
    if df[column].dtype == 'object':
        # Преобразование строковых значений в числа
        try:
            df[column] = pd.to_numeric(df[column])
        except ValueError:
            print(f"Невозможно преобразовать столбец '{column}' в числовой формат.")

# Вывод таблицы без знаков "?"
df


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1,530101,66,28,3,3,4,2
1,1,534817,88,20,0,0,4,2
2,1,530334,40,24,1,1,3,2
3,9,5290409,164,84,4,1,4,1
4,1,530255,104,35,0,0,0,2
...,...,...,...,...,...,...,...,...
295,1,533886,120,70,4,0,4,2
296,1,527702,72,24,3,2,3,1
297,1,529386,72,30,4,3,4,1
298,1,530612,100,24,3,3,3,1


In [60]:



df = pd.read_csv("D:\\Нетология python\\Статистика_в_Python\\поиск_аномалий_в_данных_и_работа_с_пропущенными_значениями\\horse_data.csv", usecols=[1,2,4,5,6,7,11,23], names=['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome'])

# Замена знаков "?" на 0
df.replace("?", 0, inplace=True)

# Вывод таблицы без знаков "?"
df



,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1,530101,66,28,3,3,4,2
1,1,534817,88,20,0,0,4,2
2,1,530334,40,24,1,1,3,2
3,9,5290409,164,84,4,1,4,1
4,1,530255,104,35,0,0,0,2
...,...,...,...,...,...,...,...,...
295,1,533886,120,70,4,0,4,2
296,1,527702,72,24,3,2,3,1
297,1,529386,72,30,4,3,4,1
298,1,530612,100,24,3,3,3,1


In [61]:
tatistics = []

for column in df.columns:
    if df[column].dtype in ['int64', 'float64']:
        sr = df[column]
        min_ = sr.min()
        max_ = sr.max()
        range_ = max_ - min_
        disp = sr.var()
        mean = sr.mean()
        median = sr.median()
        mode = int(sr.mode()[0])

        statistics.append({'Column': column, 'min_': min_, 'max_': max_, 'range_': range_, 'disp': disp, 'mean': mean, 'median': median, 'mode': mode})
    elif df[column].dtype == 'object':
        # Преобразование строковых значений в числа
        try:
            sr = pd.to_numeric(df[column])
            min_ = sr.min()
            max_ = sr.max()
            range_ = max_ - min_
            disp = sr.var()
            mean = sr.mean()
            median = sr.median()
            mode = int(sr.mode()[0])

            statistics.append({'Column': column, 'min_': min_, 'max_': max_, 'range_': range_, 'disp': disp, 'mean': mean, 'median': median, 'mode': mode})
        except ValueError:
            print(f"Невозможно преобразовать столбец '{column}' в числовой формат.")

statistics_df = pd.DataFrame(statistics)

In [62]:
statistics_df

,Column,min_,max_,range_,disp,mean,median,mode
0,surgery?,1,9,8,4.726154e+00,1.640000e+00,1.0,1
1,Age,518476,5305629,4787153,2.340291e+12,1.085889e+06,530305.5,527544
2,outcome,1,2,1,2.320959e-01,1.363333e+00,1.0,1
3,surgery?,1,9,8,4.726154e+00,1.640000e+00,1.0,1
4,Age,518476,5305629,4787153,2.340291e+12,1.085889e+06,530305.5,527544
5,rectal temperature,0,184,184,1.135807e+03,6.616000e+01,60.0,48
6,pulse,0,96,96,3.956475e+02,2.453667e+01,22.0,0
7,respiratory rate,0,4,4,1.727659e+00,1.910000e+00,2.0,3
8,temperature of extremities,0,4,4,1.559019e+00,1.553333e+00,1.0,1
9,pain,0,4,4,1.882843e+00,2.490000e+00,3.0,3
